In [ ]:
from sage.symbolic.expression_conversions import polynomial

# Round complex numbers
def round_cx(z0,n):
    return float(round(z0.real(),n)) + float(round(z0.imag(),n))*I

# The following inputs a line ax+by+cz and outputs the tuple of coefficients [a,b,c]
# Note that line.coefficients() has errors, in that a line line 3y+2z will output [3,2] rather than [0,3,2] which is what we want
def line_to_coeffs(bitangent):
    return [CC(bitangent.coefficient({x:1})), CC(bitangent.coefficient({y:1})),CC(bitangent.coefficient({z:1}))]

# Turn a list of three coefficients [a,b,c] into the polynomial ax+by+cz=0
def coeffs_to_line(list_of_coeffs):
    return x*list_of_coeffs[0] + y*list_of_coeffs[1] + z*list_of_coeffs[2]

# Provide a normalized version of the bitangent
def normalize(bitangent):
    # Base change to CC
    b = bitangent.change_ring(ComplexField(30))

    # If the x-coordinate is nonzero mod out by it
    if line_to_coeffs(b)[0] != 0:
        c = b*(1/line_to_coeffs(b)[0])
    # Else mod out by the y-coordinate
    elif line_to_coeffs(b)[1] != 0:
        c = b*(1/line_to_coeffs(b)[1])
    # Or the z-coordinate
    else:
        c = b*(1/line_to_coeffs(b)[2])
    
    # Round the output coordinates and return a new line
    new_coeffs = [round_cx(a,4) for a in line_to_coeffs(c)]
    return coeffs_to_line(new_coeffs)

def compute_normalized_bitangents(f):
    return [normalize(b) for b in compute_bitangents(f)]

# Apply a matrix M to a line B = {ax+by+cy=0}
def gp_action(M,B):
    vec = matrix([[B.coefficient({x:1})],[B.coefficient({y:1})],[B.coefficient({z:1})]])
    output_mat = (M*vec).transpose()
    w = list(list(output_mat)[0])
    outputline = w[0]*x + w[1]*y + w[2]*z
    return normalize(outputline)

We write some functions to make the equations of bitangents more readable and act on them via a subgroup $G\le \text{PGL}_3(\mathbb{C})$.

In [ ]:
# We also add some commands we'll use throughout
def matrix_to_list(M):
    return [list(r) for r in M.rows()]

# Generates a candidate list of the orbit of an input bitangent under a list of group elements
def candidate_orbit(bitangent,list_of_gp_elements):
    output_list = [bitangent]
    isotropy_gens = []
    for M in list_of_gp_elements:
        output_list.append(gp_action(M,bitangent))
        if gp_action(M,bitangent) == bitangent:
            isotropy_gens.append(M)
    for line in output_list:
        print(line)
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

def investigate_orbit(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(gp_action(group_dictionary[g],bitangent))
        if gp_action(group_dictionary[g],bitangent) == bitangent:
            isotropy_gens.append(g)
    for line in set(output_list):
        print(line)
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

def trim(bitangent):
    x_coeff = CC(bitangent.coefficient({x:1}));
    y_coeff = CC(bitangent.coefficient({y:1}));
    z_coeff = CC(bitangent.coefficient({z:1}));
    return round_cx(x_coeff,3)*x + round_cx(y_coeff,3)*y + round_cx(z_coeff,3)*z


In [ ]:
def bitan_difference(bitan1,bitan2):
    # Take the difference in absolute value between each of the coefficients and sum these
    return abs(CC(bitan1.coefficient({x:1})) - CC(bitan2.coefficient({x:1}))) + abs(CC(bitan1.coefficient({y:1})) - CC(bitan2.coefficient({y:1}))) + abs(CC(bitan1.coefficient({z:1})) - CC(bitan2.coefficient({z:1})))

# This should return some indicator of how far off two equations are numerically. Uncomment and run the following for instance:
# bitan1 =x + (-1.1234 - 4.9221*I)*y + (-2.9252 - 1.4087*I)*z
# bitan2 =x + (-1.1231 - 4.9225*I)*y + (-2.9254 - 1.4091*I)*z
# bitan_difference(bitan1,bitan2)

# Boolean to check if two bitangents agree
def same_bitangent(bitan1,bitan2, error_rate=0.01):
    if bitan_difference(bitan1,bitan2) < error_rate:
        return true
    else:
        return false

In [ ]:
def investigate_orbit(bitangent,group_dictionary):
    output_list = [bitangent]

    for g in group_dictionary.keys():
        g_bitangent = gp_action(group_dictionary[g],bitangent)

        new_bitangent = True
        for b in output_list:
            if same_bitangent(g_bitangent,b):
                new_bitangent = False

        if new_bitangent == True:
            output_list.append(g_bitangent)
    return list(set(output_list))

def investigate_isotropy(bitangent,group_dictionary):
    isotropy_gens = []
    for g in group_dictionary.keys():
        # print(g)
        g_bitangent = gp_action(group_dictionary[g],bitangent)
        # print(g_bitangent)
        if same_bitangent(g_bitangent,bitangent):
            isotropy_gens.append(g)
    return isotropy_gens